In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from rtb.datasets import get_dataset

In [12]:
dataset = get_dataset(
    name="amazon_reviews",
    root="../../data",
    category="fashion",
    use_5_core=False,
    process=True,
)

processing db...
reading product info from ../../data/amazon_reviews-fashion/db/raw/meta_AMAZON_FASHION.json...
done in 0.11 seconds.
converting to pandas dataframe...
done in 0.15 seconds.
processing product info...
done in 0.13 seconds.
reading review and customer info from ../../data/amazon_reviews-fashion/db/raw/AMAZON_FASHION.json...
done in 0.13 seconds.
converting to pandas dataframe...
done in 1.08 seconds.
processing review and customer info...
done in 0.07 seconds.
keeping only products common to product and review tables...
done in 0.28 seconds.
extracting customer table...
done in 0.11 seconds.
processing db took 2.09 seconds.
reindexing pkeys and fkeys...
reindexing pkeys and fkeys took 0.26 seconds.
saving db...
saving db took 0.50 seconds.


In [20]:
dataset.task_names

['customer_churn', 'customer_ltv']

In [14]:
db = dataset._db

In [16]:
customer = db.table_dict["customer"].df
product = db.table_dict["product"].df
review = db.table_dict["review"].df

In [17]:
customer

,customer_id,customer_name
7,0,Darrow H Ankrum II
8,1,rosieO
9,2,M. Waltman
10,3,BTDoxies
11,4,Robin Howard
...,...,...
883597,264678,Kai
883598,264679,TPalmer
883599,264680,Keith miller
883600,264681,rico


In [18]:
product

,product_id,category,brand,title,description,price
4,0,None,None,Aether & Empire #1 - 2016 First Printing Comic...,None,4.5
17,1,None,Roar Carbon,"X. L. Carbon Fiber Money Clip, made in the USA",When you pull out your extra large carbon fibe...,14.99
18,2,None,Shimmer Anne Shine,Shimmer Anne Shine Clip On Costume/Halloween C...,"A fun addition to any costume party, play, or ...",6.99
69,3,None,Buxton,Buxton Heiress Pik-Me-Up Framed Case,Authentic crunch leather with rich floral embo...,16.95
115,4,None,None,Long Blonde Witch Wig,Rubie's Costume Company has designed quality c...,9.02
...,...,...,...,...,...,...
186563,17782,None,LenzFlip,LenzFlip Polarized Replacement Lenses for Oakl...,None,30.0
186579,17783,None,Stetson,Stetson Hats Mens 10X Open Road 2 3/4 Brim 71/...,None,99.99
186589,17784,None,Ivysport,Princeton University Tigers Pennant,"Measures 12"" width by 30"" in length. Features ...",16.95
186603,17785,None,IK COLOURING,IK COLOURING Man's Automatic Mechanical Wristw...,None,52.5


In [19]:
review

,review_time,customer_id,product_id,rating,verified,review_text,summary
7,2013-09-22,0,3,3.0,True,mother - in - law wanted it as a present for h...,bought as a present
8,2013-07-17,1,3,3.0,True,"Item is of good quality. Looks great, too. But...",Buxton heiress collection
9,2013-04-13,2,3,3.0,True,I had used my last el-cheapo fake leather ciga...,Top Clasp Broke Within 3 days!
10,2013-03-09,3,3,4.0,True,This brand has been around a long time and you...,BUXTON QUALITY!
11,2013-01-27,4,3,2.0,True,I smoke 100's and these are NOT made for them....,Buxton Heiress Collection Black Leather Cigare...
...,...,...,...,...,...,...,...
883597,2017-04-05,264678,17786,5.0,True,"Great hats, thank you!",good hats!
883598,2017-03-21,264679,17786,4.0,True,Material feels very inexpensive but the hat fi...,Four Stars
883599,2017-03-15,264680,17786,5.0,True,Nice hat to wear fishing fits good.,nice hat
883600,2017-02-06,264681,17786,5.0,True,It a perfect it! made well.,Five Stars


## LTV

In [24]:
task = dataset.get_task("customer_ltv", process=True)

### train table

In [25]:
train_table = task.default_train_table

In [26]:
# here count is the number of products which contributed to the sum aggregation in ltv
train_table.df

,timestamp,customer_id,ltv,count_
0,2011-01-02,24739,0.0,0
1,2011-01-02,24740,0.0,0
2,2011-01-02,24741,0.0,0
3,2011-01-02,24742,0.0,0
4,2011-01-02,24868,0.0,0
...,...,...,...,...
936,2009-01-02,202902,0.0,0
937,2009-01-02,202912,0.0,0
938,2011-01-02,197668,0.0,0
939,2011-01-02,197582,0.0,0


In [28]:
# frequency of counts in percent
train_table.df["count_"].value_counts(normalize=True) * 100

count_
0    99.68119
1     0.31881
Name: proportion, dtype: float64

## val table

In [29]:
val_table = task.default_val_table

In [30]:
val_table.df

,timestamp,customer_id,ltv,count_
0,2013-01-01,204816,0.0,0
1,2013-01-01,204842,0.0,0
2,2013-01-01,204845,0.0,0
3,2013-01-01,204872,0.0,0
4,2013-01-01,204889,0.0,0
...,...,...,...,...
3545,2013-01-01,198655,0.0,0
3546,2013-01-01,197560,0.0,0
3547,2013-01-01,197591,0.0,0
3548,2013-01-01,199706,0.0,0


In [32]:
val_table.df["count_"].value_counts(normalize=True) * 100

count_
0    98.788732
1     1.014085
2     0.197183
Name: proportion, dtype: float64

## Churn

In [35]:
task = dataset.get_task("customer_churn", process=True)
task

CustomerChurnTask(dataset=AmazonReviewsDataset())

### train table

In [36]:
train_table = task.default_train_table

In [38]:
train_table.df

,timestamp,customer_id,churn
0,2011-01-02,12564,1
1,2011-01-02,12565,1
2,2011-01-02,12566,1
3,2011-01-02,12567,1
4,2011-01-02,12568,1
...,...,...,...
936,2011-01-02,204543,1
937,2011-01-02,204596,1
938,2009-01-02,202864,1
939,2009-01-02,202902,1


In [39]:
# frequency of churn in percent
train_table.df["churn"].value_counts(normalize=True) * 100

churn
1    99.68119
0     0.31881
Name: proportion, dtype: float64

## val table

In [41]:
val_table = task.default_val_table

In [42]:
val_table.df

,timestamp,customer_id,churn
0,2013-01-01,6,1
1,2013-01-01,96,1
2,2013-01-01,97,1
3,2013-01-01,98,1
4,2013-01-01,99,1
...,...,...,...
3545,2013-01-01,198226,1
3546,2013-01-01,198332,1
3547,2013-01-01,198581,1
3548,2013-01-01,198582,1


In [43]:
val_table.df["churn"].value_counts(normalize=True) * 100

churn
1    98.788732
0     1.211268
Name: proportion, dtype: float64